In [1]:
import heapq
import pygame
import random
import os
import math
import time
from random import randint
from ast import literal_eval as make_tuple

In [2]:
#Convention
# Initialize Grid
# 0 is blocked and 1 is unblocked for maze
# 0 is unvisited and 1 is visited for track_maze
# gn = get neighbour
# nn = new neighbour

In [3]:
# Initialize Screen
pygame.init()
myfont = pygame.font.SysFont("monospace", 15)
blockwidth = 8  # Drawing dimensions of block
GridCols = 101
GridRows = 101


In [4]:
maze = [[0 for y in range(GridRows)] for x in range(GridCols)]
track_maze = [[0 for y in range(GridRows)] for x in range(GridCols)]
start_x = 0
start_y = 0
last_x = 0
last_y = 0
cursor_x = 0
cursor_y = 0

In [5]:
tree = {}
tree["5,5"] = ["5,4","5,6"]
tree["4,4"] = ["5,5"]
print(tree)
tree["5,5"].append("6,5")
print(tree)

{'5,5': ['5,4', '5,6'], '4,4': ['5,5']}
{'5,5': ['5,4', '5,6', '6,5'], '4,4': ['5,5']}


In [6]:
def get_key(value):
    key_value_dict = {}
    for i in tree.items():
        a = i[0]
        b = i[1]
        print(a,b)
        if a and b:
            for item in b:
                key_value_dict[str(item)] = str(a)
    print("i am here2")
    print(key_value_dict,value)
    return key_value_dict[value]

In [7]:
#get_key("37,97")


In [27]:
#get the neighbours for a cell.
def get_neighbour(x,y,point): 
    try:
        if point == 0:
            if x-1> 0 and y-1 > 0:
                return x-1,y-1
        if point == 1:
            if x-1 > 0 and y > 0:
                return x-1,y
        if point == 2:
            if x-1 > 0 and y+1 < 101:
                return x-1,y+1
        if point == 3:
            if x > 0 and y-1 > 0:
                return x,y-1
        if point == 4:
            if x > 0 and y+1 < 101:
                return x,y+1
        if point == 5:
            if x+1 < 101 and y-1 > 0:
                return x+1,y-1
        if point == 6:
            if x+1 < 101 and y > 0:
                return x+1,y
        if point == 7:
            if x+1 <101 and y+1 < 101:
                return x+1,y+1
    except:
        return None,None
    return None,None

#check if the cell has neighbours that are not visited.
def validate_neighbour(x,y):
    tf_value = False
    try: 
        if track_maze[x-1][y-1] == 0:
            tf_value = True
    except:
        pass    
        
    try:
        if track_maze[x-1][y] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if track_maze[x-1][y+1] == 0:
            tf_value = True
    except:
        pass
    
    try:
        if track_maze[x][y-1] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if track_maze[x][y+1] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if track_maze[x+1][y-1] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if track_maze[x+1][y] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if track_maze[x+1][y+1] == 0:
            tf_value = True
    except:
        pass
    return tf_value

def get_validate_neighbour(x,y):
    x = int(x)
    y = int(y)
    try: 
        if track_maze[x-1][y-1] == 0:
            print(1)
            return x-1, y-1
    except:
        pass
    
    try:
        if track_maze[x-1][y] == 0:
            return x-1, y
    except:
        pass
    
    try:
        if track_maze[x-1][y+1] == 0:
            return x-1, y+1
    except:
        pass
    
    try:
        if track_maze[x][y-1] == 0:
            return x, y-1
    except:
        pass
    
    try:
        if track_maze[x][y+1] == 0:
            return x, y+1
    except:
        pass
    
    try:
        if track_maze[x+1][y-1] == 0:
            return x+1, y-1
    except:
        pass
    
    try:
        if track_maze[x+1][y] == 0:
            return x+1, y
    except:
        pass
    
    try:
        if track_maze[x+1][y+1] == 0:
            return x+1, y+1
    except:
        pass
    
    return None, None

#get the string for the cell location
def get_String(x,y):
    return str(x)+","+str(y)

def get_coord(cell):
    li = cell.split(",")
    return int(li[0]),int(li[1])

#check if the cell is visited or not.
def is_visited(x,y):
    if track_maze[x][y] == 1:
        return True
    else:
        return False

#Call to build the tree, backtrack and then call again this recursively.
def build_tree(p_x,p_y):
    last_x = None
    last_y = None
    print("Initially",p_x,p_y)
    nn_x,nn_y = get_neighbour(p_x,p_y,randint(0,7))
    while validate_neighbour(p_x,p_y) is True:
        if nn_x == None or nn_y == None or is_visited(nn_x,nn_y):
            while True: 
                if nn_x != None and nn_y != None and is_visited(nn_x,nn_y) == False:
                    print("I am here1",nn_x,nn_y)
                    print(p_x,p_y)
                    print("-----")
                    break
                nn_x,nn_y = get_neighbour(p_x,p_y,randint(0,7))
                
        if nn_x is not None and nn_y is not None and not is_visited(nn_x,nn_y):
            try:
                tree[str(p_x)+","+str(p_y)] = tree[str(p_x)+","+str(p_y)].append(get_String(nn_x,nn_y))
            except:
                tree[str(p_x)+","+str(p_y)] = [get_String(nn_x,nn_y)]
            rand_block_unblock = randint(0,100)
            if rand_block_unblock < 30:
                maze[p_x][p_y] = 1 # to be changed to 0, using to track
            else:
                maze[p_x][p_y] = 1
            track_maze[p_x][p_y] = 1
            p_x,p_y = nn_x,nn_y
            nn_x,nn_y = get_neighbour(p_x,p_y,randint(0,7))
            last_x = p_x
            last_y = p_y
    return last_x, last_y

def is_first(x,y):
    if x == start_x and y == start_y:
        return True
    else:
        return False

def back_track(last_x,last_y):
    print(last_x,last_y)
    if validate_neighbour(last_x,last_y):
            return get_String(last_x,last_y)
        
    while is_first(last_x,last_y) == False:
        parent = get_key(get_String(last_x,last_y))
        x,y = get_coord(parent)
        if validate_neighbour(x,y):
            #use to get the neighbour get_validate_neighbour(x,y)
            nnn_x,nnn_y = get_validate_neighbour(x,y)
            if nnn_x is not None and nnn_y is not None:
                print("backtrack",get_String(nnn_x,nnn_y))
                return get_String(nnn_x,nnn_y)
        last_x,last_y = x,y


    

In [28]:
print(validate_neighbour(90,65))

True


In [10]:
# Make Random Grid
def makeGrid():
    i = 1
    gn_x = None
    gn_y = None
    p_x = None
    p_y = None
    while True:
        if gn_x != None and gn_y != None:
            print("first")
            break
        p_x = randint(0,GridCols-1)
        p_y = randint(0,GridRows-1)
        start_x = p_x
        start_y = p_y
        gn_x,gn_y = get_neighbour(p_x,p_y,randint(0,7)) #get neighbor
   
    maze[p_x][p_y] = 1
    track_maze[p_x][p_y] = 1
    tree[str(p_x)+","+str(p_y)] = [get_String(gn_x,gn_y)]
    p_x,p_y = gn_x,gn_y
    # start building the tree. when deadend is reach call the bcaktrack and call again the build tree.
    #while(get_backtrack_cell is not None)
    #call build_tree for the backtrack_cell.
    while True:
        i = i + 1 
        if i == 2000:
            break
        last_x,last_y = build_tree(p_x,p_y)
        if last_x and last_y:
            new_parent = back_track(last_x,last_y)
            if new_parent:
                new_px,new_py = get_coord(new_parent)
                p_x,p_y = new_px,new_py
                if p_x == start_x and p_y == start_y:
                    break
            else:
                print(new_parent)
                break
        else:
            print(last_x,last_y)
            break
        print(tree)
        
    return None


In [11]:
#makeGrid()

In [12]:
# Generate Start and Finish
def generateStartFinish():
	# Generate Start
	x = randint(0,39)
	y = randint(0,39)
	if x>20:
		x = GridCols-x+20
	if y>20:
		y = GridRows-y+20
		
	while maze[x][y]=='a' or maze[x][y]=='b' or maze[x][y]=='0':
		x = randint(0,39)
		y = randint(0,39)
		if x>20:
			x = GridCols-x+20
		if y>20:
			y = GridRows-y+20
		
	start_x = x
	start_y = y
	
	# Generate Finish
	while maze[x][y]=='a' or maze[x][y]=='b' or maze[x][y]=='0' or math.sqrt((x-start_x)**2+(y-start_y)**2)<100:
		x = randint(0,39)
		y = randint(0,39)
		if x>20:
			x = GridCols-x+20
		if y>20:
			y = GridRows-y+20

	goal_x = x
	goal_y = y
	
	return start_x,start_y,goal_x,goal_y


In [13]:
# Draw Grid
def drawGrid(myGridSurface):
	myGridSurface.fill((255,255,255))
	for x in range(len(maze)):
		for y in range(len(maze[x])):
			if maze[x][y] == 0: 
				pygame.draw.rect(myGridSurface, (40,40,40), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
				pygame.draw.rect(myGridSurface, (40,40,40), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
			elif maze[x][y] == 1: 
				pygame.draw.rect(myGridSurface, (255,255,255), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
				pygame.draw.rect(myGridSurface, (100,100,100), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
			elif maze[x][y] == '2': 
				pygame.draw.rect(myGridSurface, (200,200,200), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
				pygame.draw.rect(myGridSurface, (100,100,100), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
			elif maze[x][y] == 'a': 
				pygame.draw.rect(myGridSurface, (130,170,255), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
				pygame.draw.rect(myGridSurface, (100,100,100), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
			elif maze[x][y] == 'b': 
				pygame.draw.rect(myGridSurface, (70,90,220), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
				pygame.draw.rect(myGridSurface, (100,100,100), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
	
	myGridSurface = myGridSurface.convert()
	return myGridSurface
	


In [14]:
# Draw Screen
def drawScreen(GridSurface,closed_list,path,pathcost,nodes_expanded,mode,elapsedtime,fn_g,fn_f,fn_h):
	
	GameScreen.blit(GridSurface,(0,0))
	
	#pygame.draw.circle(GameScreen, (255,0,0), int(start_x*blockwidth+blockwidth/2+10,start_y*blockwidth+blockwidth/2+10),blockwidth/2, 0)
	#pygame.draw.circle(GameScreen, (0,0,255), int(goal_x*blockwidth+blockwidth/2+10,goal_y*blockwidth+blockwidth/2+10),blockwidth/2, 0)
	
	pygame.draw.rect(GameScreen, (255,0,0), (cursor_x*blockwidth+9,cursor_y*blockwidth+9,blockwidth+2,blockwidth+2), 2)
	
	# Draw text
	label = myfont.render("G = New Grid | E = New Endpoints | U = Uniform Cost | A = A* Search | W = Weighted A* | V = Visited | S = Save | L = Load", 1, (0,0,0))
	GameScreen.blit(label, (20, blockwidth*GridRows+14))
	
	#label = myfont.render("Cell: ("+str(cursor_x)+","+str(cursor_y)+")", 1, (0,0,0))
	#GameScreen.blit(label, (10+blockwidth*GridCols+30, 30))
	
	#label = myfont.render("Type: "+maze[cursor_x][cursor_y], 1, (0,0,0))
	#GameScreen.blit(label, (10+blockwidth*GridCols+30, 50))
	
	
	label = myfont.render("Start: ("+str(start_x)+","+str(start_y)+")", 1, (0,0,0))
	GameScreen.blit(label, (10+blockwidth*GridCols+30, 80))
	label = myfont.render("Goal: ("+str(goal_x)+","+str(goal_y)+")", 1, (0,0,0))
	GameScreen.blit(label, (10+blockwidth*GridCols+30, 100))
	
	if pathcost != 0:
		label = myfont.render("Path Cost:", 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 130))
		label = myfont.render(str(pathcost), 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 150))
	
	if pathcost != 0:
		label = myfont.render("Nodes Expanded:", 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 170))
		label = myfont.render(str(nodes_expanded), 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 190))
	
	if (cursor_x,cursor_y) in fn_f:
		label = myfont.render("f: "+str(fn_f[(cursor_x,cursor_y)]), 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 220))
	
	if (cursor_x,cursor_y) in fn_g:
		label = myfont.render("g: "+str(fn_g[(cursor_x,cursor_y)]), 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 240))
	
	if (cursor_x,cursor_y) in fn_h:
		label = myfont.render("h: "+str(fn_h[(cursor_x,cursor_y)]), 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, 260))
	
	label = myfont.render("Time: ", 1, (0,0,0))
	GameScreen.blit(label, (10+blockwidth*GridCols+30, 290))
	label = myfont.render(str(elapsedtime*1000)+" ms", 1, (0,0,0))
	GameScreen.blit(label, (10+blockwidth*GridCols+30, 310))
	
	label = myfont.render("Neighbors:", 1, (0,0,0))
	GameScreen.blit(label, (10+blockwidth*GridCols+30, 340))
	
	draw_y = 360
	neighbors = getNeighbors(cursor_x,cursor_y)
	
	for neighbor in neighbors:
		label = myfont.render("("+str(neighbor[0])+","+str(neighbor[1])+")", 1, (0,0,0))
		GameScreen.blit(label, (10+blockwidth*GridCols+30, draw_y))
		draw_y += 20
	
	# Draw Final Path and Closed List
	if mode == 1:
		for cell in closed_list:
			pygame.draw.circle(GameScreen, (0,255,0), (cell[0]*blockwidth+blockwidth/2+10,cell[1]*blockwidth+blockwidth/2+10),blockwidth/4, 0)
	for cell in path:
		pygame.draw.circle(GameScreen, (255,0,0), (cell[0]*blockwidth+blockwidth/2+10,cell[1]*blockwidth+blockwidth/2+10),blockwidth/4, 0)
	
	pygame.display.update()


In [15]:
# Get neighbors of a cell
def getNeighbors(x,y):
	myneighbors = [(x-1,y-1),(x,y-1),(x+1,y-1),(x-1,y),(x+1,y),(x-1,y+1),(x,y+1),(x+1,y+1)]
	myneighbors[:] = [neighbor for neighbor in myneighbors if neighbor[0]>=0 and neighbor[1]>=0 and neighbor[0]<GridCols and neighbor[1]<GridRows and maze[neighbor[0]][neighbor[1]] != '0']
	
	return myneighbors

def cost(currx, curry, nextx, nexty):
	cost = 0.0
	
	if (maze[currx][curry] == 'a' or maze[currx][curry] == '1') and (maze[nextx][nexty] == 'a' or maze[nextx][nexty] == '1'):
		if((currx == nextx - 1 or currx == nextx + 1) and (curry == nexty - 1 or curry == nexty + 1)):
			#traversing unblocked diagonally
			cost = math.sqrt(2)
		else:
			#traversing unblocked diagonally
			cost = 1
	if (maze[currx][curry] == 'b' or maze[currx][curry] == '2') and (maze[nextx][nexty] == '2' or maze[nextx][nexty] == 'b'):
		if((currx == nextx - 1 or currx == nextx + 1) and (curry == nexty - 1 or curry == nexty + 1)):
			#traversing unblocked diagonally
			cost = math.sqrt(8)
		else:
			#traversing unblocked diagonally
			cost = 2
	if (maze[currx][curry] == 'a' or maze[currx][curry] == '1') and (maze[nextx][nexty] == '2' or maze[nextx][nexty] == 'b'):
		if((currx == nextx - 1 or currx == nextx + 1) and (curry == nexty - 1 or curry == nexty + 1)):
			#traversing unblocked diagonally
			cost = (math.sqrt(2) + math.sqrt(8)) / 2
		else:
			#traversing unblocked diagonally
			cost = 1.5
	if (maze[currx][curry] == '2' or maze[currx][curry] == 'b') and (maze[nextx][nexty] == '1' or maze[nextx][nexty] == 'a'):
		if((currx == nextx - 1 or currx == nextx + 1) and (curry == nexty - 1 or curry == nexty + 1)):
			#traversing unblocked diagonally
			cost = (math.sqrt(2) + math.sqrt(8)) / 2
		else:
			#traversing unblocked diagonally
			cost = 1.5
	if (maze[currx][curry] == 'a' or maze[currx][curry] == 'b') and (maze[nextx][nexty] == 'a' or maze[nextx][nexty] == 'b') and (((currx == nextx + 1 or currx == nexty - 1) and curry == nexty) or ((curry == nexty + 1 or curry == nexty - 1) and currx == nextx)):
		cost = cost / 4
		return cost
	if cost == 0.0:
		print("Could not find cost value. Current: " + str(currx) + ", " + str(curry) + " of type: " + maze[currx][curry] + "/Next: " + str(nextx) + ", " + str(nexty) + " of type: " + maze[nextx][nexty])


	return cost


In [16]:
# A* Stuff

class Coordinate:
	def __init__(self, x, y, parent):
		self.x = x
		self.y = y
		self.parent = parent
		
	def get_x(self):
		return self.x
		
	def get_y(self):
		return self.y
	
	def get_parent(self):
		return self.parent

class PriorityQueue:
	def __init__(self):
		self.elements = []

	def empty(self):
		return len(self.elements) == 0

	def put(self, item, priority):
		heapq.heappush(self.elements, (priority, item))

	def get(self):
		return heapq.heappop(self.elements)[1]

def heuristic(self,startx,starty,goalx,goaly,choice):
	start = (startx, starty)
	goal = (goalx, goaly)
	choice = int(choice)

	if choice == 1: #manhattan
			heuristic = abs(int(startx) - int(goalx)) + abs(int(starty) - int(goaly))
			heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
			return heuristic
	if choice == 2: #euclidean
			heuristic = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
			heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
			return heuristic
	if choice == 3: #octile
			dx = abs(int(startx) - int(goalx))
			dy = abs(int(starty) - int(goaly))
			heuristic = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
			heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
			return heuristic
	if choice == 4: #Chebyshev
			heuristic = max(abs(startx - goalx), abs(starty - goaly))
			heuristic *= (1.0 + (0.25/160))
			return heuristic
	if choice == 5: #5th heuristic
			heuristic = math.sqrt(2)*min(abs(startx - goalx), abs(starty - goaly)) + max(abs(startx - goalx), abs(starty - goaly)) - min(abs(startx - goalx), abs(starty - goaly))
			heuristic *= (1.0 + (0.25/160))
			return heuristic
	if choice == 6: #Best - minimum of all
			dx = abs(int(startx) - int(goalx))
			dy = abs(int(starty) - int(goaly))
			h1 = dx + dy 
			h2 = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
			h3 = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
			h4 = max(dx, dy)
			h5 = math.sqrt(2)*min(dx, dy) + max(dx, dy) - min(dx, dy)
			h6 = min(h1, h2, h3, h4, h5)
			h6 *= (1.0 + (0.25/160))
			return h6
	return 0


In [17]:
class AStarSearch(object):
	def Search(self, startx, starty, goalx, goaly, choice):
		fringe = PriorityQueue()
		start = Coordinate(startx, starty, None)
		goal = (goalx, goaly)
		fringe.put(start, 0)

		closed_list = {}
		cost_added = {}
		final_path = []
		heuristic_list = {}
		priority_list = {}
		closed_list[(start.get_x(),start.get_y())] = None
		cost_added[(start.get_x(),start.get_y())] = 0

		# f = priority_list[]
		# g = cost_added[]
		# h = heuristic
		
		while not fringe.empty():
			current = fringe.get()
			#print "got current from fringe with x %d and y %d" % current[0], current[1]
			if (current.get_x(),current.get_y()) == goal:
				print("Made it to goal at " + str(goal[0]) + "," + str(goal[1]))
				
				# Make a straight path from goal to start
				PathNode = current
				
				while PathNode != None:
					final_path.append((PathNode.get_x(),PathNode.get_y()))
					#print "Path: " + str(current.get_x()) + "," + str(current.get_y())
					PathNode = PathNode.get_parent()
			
				break

			for next in getNeighbors(current.get_x(), current.get_y()):
				#print "current x %d current y %d" % current[0], current[1]
				new_cost = cost_added[(current.get_x(),current.get_y())] + cost(current.get_x(), current.get_y(), next[0], next[1])
				
				if next not in cost_added or new_cost < cost_added[next]:
				#if next not in closed_list or new_cost < cost_added[next]:
					cost_added[next] = new_cost		# g
					myheuristic = self.heuristic(next[0], next[1], goal_x, goal_y, choice)
					priority = new_cost + myheuristic
					heuristic_list[next] = myheuristic
					priority_list[next] = priority
					fringe.put(Coordinate(next[0],next[1],current), priority)
					closed_list[next] = current
		
		return closed_list, cost_added, final_path, cost_added[(goalx,goaly)], priority_list, heuristic_list

	def heuristic(self,startx,starty,goalx,goaly,choice):
		start = (startx, starty)
		goal = (goalx, goaly)
		choice = int(choice)

		if choice == 1: #manhattan
				heuristic = abs(int(startx) - int(goalx)) + abs(int(starty) - int(goaly))
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 2: #euclidean
				heuristic = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 3: #octile
				dx = abs(int(startx) - int(goalx))
				dy = abs(int(starty) - int(goaly))
				heuristic = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 4: #Chebyshev
				heuristic = max(abs(startx - goalx), abs(starty - goaly))
				heuristic *= (1.0 + (0.25/160))
				return heuristic
		if choice == 5: #5th heuristic
				heuristic = math.sqrt(2)*min(abs(startx - goalx), abs(starty - goaly)) + max(abs(startx - goalx), abs(starty - goaly)) - min(abs(startx - goalx), abs(starty - goaly))
				heuristic *= (1.0 + (0.25/160))
				return heuristic
		if choice == 6: #Best - minimum of all
				dx = abs(int(startx) - int(goalx))
				dy = abs(int(starty) - int(goaly))
				h1 = dx + dy 
				h2 = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
				h3 = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
				h4 = max(dx, dy)
				h5 = math.sqrt(2)*min(dx, dy) + max(dx, dy) - min(dx, dy)
				h6 = min(h1, h2, h3, h4, h5)
				h6 *= (1.0 + (0.25/160))
				return h6
		return 0
class AStarSearch(object):
	def Search(self, startx, starty, goalx, goaly, choice):
		fringe = PriorityQueue()
		start = Coordinate(startx, starty, None)
		goal = (goalx, goaly)
		fringe.put(start, 0)

		closed_list = {}
		cost_added = {}
		final_path = []
		heuristic_list = {}
		priority_list = {}
		closed_list[(start.get_x(),start.get_y())] = None
		cost_added[(start.get_x(),start.get_y())] = 0

		# f = priority_list[]
		# g = cost_added[]
		# h = heuristic
		
		while not fringe.empty():
			current = fringe.get()
			#print "got current from fringe with x %d and y %d" % current[0], current[1]
			if (current.get_x(),current.get_y()) == goal:
				print("Made it to goal at " + str(goal[0]) + "," + str(goal[1]))
				
				# Make a straight path from goal to start
				PathNode = current
				
				while PathNode != None:
					final_path.append((PathNode.get_x(),PathNode.get_y()))
					#print "Path: " + str(current.get_x()) + "," + str(current.get_y())
					PathNode = PathNode.get_parent()
			
				break

			for next in getNeighbors(current.get_x(), current.get_y()):
				#print "current x %d current y %d" % current[0], current[1]
				new_cost = cost_added[(current.get_x(),current.get_y())] + cost(current.get_x(), current.get_y(), next[0], next[1])
				
				if next not in cost_added or new_cost < cost_added[next]:
				#if next not in closed_list or new_cost < cost_added[next]:
					cost_added[next] = new_cost		# g
					myheuristic = self.heuristic(next[0], next[1], goal_x, goal_y, choice)
					priority = new_cost + myheuristic
					heuristic_list[next] = myheuristic
					priority_list[next] = priority
					fringe.put(Coordinate(next[0],next[1],current), priority)
					closed_list[next] = current
		
		return closed_list, cost_added, final_path, cost_added[(goalx,goaly)], priority_list, heuristic_list

	def heuristic(self,startx,starty,goalx,goaly,choice):
		start = (startx, starty)
		goal = (goalx, goaly)
		choice = int(choice)

		if choice == 1: #manhattan
				heuristic = abs(int(startx) - int(goalx)) + abs(int(starty) - int(goaly))
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 2: #euclidean
				heuristic = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 3: #octile
				dx = abs(int(startx) - int(goalx))
				dy = abs(int(starty) - int(goaly))
				heuristic = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
				heuristic *= (1.0 + (0.25/160)) #tie-breaker by multiplying the heuristic by (minimum cost)/(max possible path length)
				return heuristic
		if choice == 4: #Chebyshev
				heuristic = max(abs(startx - goalx), abs(starty - goaly))
				heuristic *= (1.0 + (0.25/160))
				return heuristic
		if choice == 5: #5th heuristic
				heuristic = math.sqrt(2)*min(abs(startx - goalx), abs(starty - goaly)) + max(abs(startx - goalx), abs(starty - goaly)) - min(abs(startx - goalx), abs(starty - goaly))
				heuristic *= (1.0 + (0.25/160))
				return heuristic
		if choice == 6: #Best - minimum of all
				dx = abs(int(startx) - int(goalx))
				dy = abs(int(starty) - int(goaly))
				h1 = dx + dy 
				h2 = math.sqrt(((int(startx) - int(goalx)) ** 2) + ((int(starty) - int(goaly)) ** 2))
				h3 = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
				h4 = max(dx, dy)
				h5 = math.sqrt(2)*min(dx, dy) + max(dx, dy) - min(dx, dy)
				h6 = min(h1, h2, h3, h4, h5)
				h6 *= (1.0 + (0.25/160))
				return h6
		return 0
				
class UniformCostSearch(AStarSearch):
	def heuristic(self,startx, starty, goalx, goaly, choice):
		return 1

class WeightedAStarSearch(AStarSearch):
	def __init__(self,weight):
		self.weight = weight
	def heuristic(self,startx, starty, goalx, goaly, choice):
		return super(WeightedAStarSearch, self).heuristic(startx,starty,goalx,goaly,choice) * float(self.weight)

#Phase 2 Begin
class SequentialAStarSearch(AStarSearch):
	def Search(self,startx, starty, goalx, goaly, notused):
		fringe = [PriorityQueue() for x in range(5)]
		start = Coordinate(startx, starty, None)
		goal = (goalx, goaly)

		closed_list = [dict() for y in range (0,5)]
		cost_added = [dict() for y in range (0,5)]
		heuristic_list = [dict() for y in range(0,5)]
		final_path = []
		priority_list = [dict() for y in range(0,5)]
		w2 = 1 #weight
		path_cost = 0
		'''closed_list = {}
		cost_added = {}
		final_path = []
		heuristic_list = {}
		priority_list = {}
		closed_list[(start.get_x(),start.get_y())] = None
		cost_added[(start.get_x(),start.get_y())] = 0
		'''
		for i in range(0, 5):
			closed_list[i] = {}
			cost_added[i] = {}
			heuristic_list[i] = {}
			for next in getNeighbors(start.get_x(), start.get_y()):
				#print "current x %d current y %d" % current[0], current[1]
				new_cost = cost(start.get_x(), start.get_y(), next[0], next[1])
				if next not in cost_added[i] or new_cost < cost_added[i][next]:
					#if next not in closed_list or new_cost < cost_added[next]:
					cost_added[i][next] = new_cost		# g
					myheuristic = self.heuristic(next[0], next[1], goal_x, goal_y, i+1)
					priority = new_cost + myheuristic
					heuristic_list[i][next] = myheuristic
					priority_list[i][next] = priority
					fringe[i].put(Coordinate(next[0],next[1],start), priority)
					closed_list[i][next] = start
			closed_list[i][(start.get_x(), start.get_y())] = None
			cost_added[i][(start.get_x(), start.get_y())] = 0
			cost_added[i][(goalx, goaly)] = float("inf")
		# f = priority_list[]
		# g = cost_added[]
		# h = heuristic
		while not fringe[0].empty():
			anchor = fringe[0].getFull()
			for i in range(1, 5):
				#anchor is a tuple (item, priority)
				temp = fringe[i].getFull() #temp is a tuple (item, priority)
				
				'''if temp != -1:
					print "x,y of anchor is: ("+str(anchor[0].get_x())+","+str(anchor[0].get_y())+")"
					print "x,y of temp is: (" + str(temp[0].get_x()) + ","+str(temp[0].get_y())+")"
					print "priority/key of anchor is: " + str(anchor[1])
					print "priority/key of temp is: " + str(temp[1])'''

				#[0] = item
				#[1] = priority

				if temp!= -1 and temp[1] <= w2*anchor[1]: #main condition
					if temp[0].get_x() == goalx and temp[0].get_y() == goaly:
						# Found goal, return path
						print("Made it to goal at " + str(goal[0]) + "," + str(goal[1]))
						path_cost = 0
						# Make a straight path from goal to start
						PathNode = temp[0]
						old_x = PathNode.get_x()
						old_y = PathNode.get_y()
						final_path.append((old_x,old_y))
						PathNode = PathNode.get_parent()

						while PathNode != None:
							path_cost += cost(PathNode.get_x(),PathNode.get_y(),old_x,old_y)
							final_path.append((PathNode.get_x(),PathNode.get_y()))
							#print "Path: " + str(current.get_x()) + "," + str(current.get_y())
							old_x = PathNode.get_x()
							old_y = PathNode.get_y()
							PathNode = PathNode.get_parent()
						
						#break
						return closed_list, cost_added, final_path, path_cost, priority_list, heuristic_list
					else:
						#print "adding to fringe[" + str(i)+"]"
						for next in getNeighbors(temp[0].get_x(), temp[0].get_y()):
							#print "current x %d current y %d" % current[0], current[1]
							new_cost = cost_added[i][(temp[0].get_x(),temp[0].get_y())] + cost(temp[0].get_x(), temp[0].get_y(), next[0], next[1])

							if next not in cost_added[i] or new_cost < cost_added[i][next]:
								#print "Added neighbor"
								#if next not in closed_list or new_cost < cost_added[next]:
								cost_added[i][next] = new_cost		# g
								myheuristic = self.heuristic(next[0], next[1], goalx, goaly, i+1)
								priority = new_cost + myheuristic
								heuristic_list[i][next] = myheuristic
								priority_list[i][next] = priority
								fringe[i].put(Coordinate(next[0],next[1],temp[0]), priority)
								closed_list[i][next] = temp[0]

				else:
					#print "------------- Reached the else condition, using fringe[0] ---------------"
					if anchor[0].get_x() == goalx and anchor[0].get_y() == goaly:
						# Found goal, return path
						print("Made it to goal at" + str(goal[0]) + "," + str(goal[1]))
						path_cost = 0
						# Make a straight path from goal to start
						PathNode = anchor[0]
						old_x = PathNode.get_x()
						old_y = PathNode.get_y()
						final_path.append((old_x,old_y))
						PathNode = PathNode.get_parent()

						while PathNode != None:
							path_cost += cost(PathNode.get_x(),PathNode.get_y(),old_x,old_y)
							final_path.append((PathNode.get_x(),PathNode.get_y()))
							#print "Path: " + str(current.get_x()) + "," + str(current.get_y())
							old_x = PathNode.get_x()
							old_y = PathNode.get_y()
							PathNode = PathNode.get_parent()
						
						#break
						return closed_list, cost_added, final_path, path_cost, priority_list, heuristic_list
					else:
						#print "-------------- adding to fringe[0] ---------------"
						for next in getNeighbors(anchor[0].get_x(), anchor[0].get_y()):
							#print "current x %d current y %d" % current[0], current[1]
							new_cost=cost_added[0][(anchor[0].get_x(), anchor[0].get_y())]+cost(anchor[0].get_x(),anchor[0].get_y(),next[0],next[1])

							if next not in cost_added[0] or new_cost < cost_added[0][next]:
								#print "Added neighbor"
							#if next not in closed_list or new_cost < cost_added[next]:
								cost_added[0][next] = new_cost		# g
								myheuristic = self.heuristic(next[0], next[1], goalx, goaly, 1)
								priority = new_cost + myheuristic
								heuristic_list[0][next] = myheuristic
								priority_list[0][next] = priority
								fringe[0].put(Coordinate(next[0],next[1],anchor[0]), priority)
								closed_list[0][next] = anchor[0]


		return closed_list, cost_added, final_path, path_cost, priority_list, heuristic_list

	def heuristic(self,startx,starty,goalx,goaly,choice):
		super(SequentialAStarSearch, self).heuristic(startx,starty,goalx,goaly,choice)



In [18]:
# Main loop
running = True
GameScreen = pygame.display.set_mode((GridCols*blockwidth+200,GridRows*blockwidth+34))
GridSurface = pygame.Surface(GameScreen.get_size())

areacoordinates = makeGrid()
time.sleep(2)
GridSurface = drawGrid(GridSurface)
start_x,start_y,goal_x,goal_y = generateStartFinish()
final_path = []
closed_list = []
cell_costs = []
priority_list = []
heuristic_list = []
path_cost = 0
elapsed_time = 0
drawmode = 0
nodes_expanded = 0

MySearch = AStarSearch() # Initialize Object

while(running):
	# Get Input
	for event in pygame.event.get():
		if event.type == pygame.QUIT:
			running = False
		elif event.type == pygame.KEYDOWN:
			if event.key == pygame.K_ESCAPE:
				running = False
			elif event.key == pygame.K_g:
				maze = [['1' for y in range(GridRows)] for x in range(GridCols)]
				areacoordinates = makeGrid()
				GridSurface = drawGrid(GridSurface)
				start_x,start_y,goal_x,goal_y = generateStartFinish()
				final_path = []
				closed_list = []
				priority_list = []
				heuristic_list = []
				cell_costs = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				print("Generated new map")
			elif event.key == pygame.K_e:
				start_x,start_y,goal_x,goal_y = generateStartFinish()
				final_path = []
				closed_list = []
				priority_list = []
				heuristic_list = []
				cell_costs = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				print("Generated new start and finish points")
			elif event.key == pygame.K_s:
				# Save map: get filename
				filename = raw_input("Save map to: ")
				with open(os.path.join("./gen",filename),"w") as mapfile:
					mapfile.write(str((start_x,start_y)))		# Write start
					mapfile.write("\n")
					mapfile.write(str((goal_x,goal_y)))			# Write goal
					mapfile.write("\n")
					
					for area in areacoordinates:				# Write hard to traverse area centers
						mapfile.write(str((area[0],area[1])))		
						mapfile.write("\n")
					
					for y in range(len(maze[x])):				# Write each cell
						for x in range(len(maze)):					
							mapfile.write(str(maze[x][y]))		
						mapfile.write("\n")
					
					mapfile.close()
				print("Map saved!")
			elif event.key == pygame.K_l:
				# Load map: get filename
				filename = raw_input("Load map from: ")
				with open(os.path.join("./gen",filename),"r") as mapfile: #changed to allow using /maps folder
					new_start = make_tuple(mapfile.readline())
					start_x = new_start[0]
					start_y = new_start[1]
					new_goal = make_tuple(mapfile.readline())
					goal_x = new_goal[0]
					goal_y = new_goal[1]
					
					areacoordinates = []
					
					for i in range(8):
						new_area = make_tuple(mapfile.readline())
						areacoordinates.append((new_area[0],new_area[1]))
					
					for y in range(len(maze[x])):				# Read each cell
						for x in range(len(maze)):					
							maze[x][y] = mapfile.read(1)
						mapfile.read(1)
					
					mapfile.close()
				final_path = []
				closed_list = []
				cell_costs = []
				priority_list = []
				heuristic_list = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				GridSurface = drawGrid(GridSurface)
				print("Map loaded!")
			elif event.key == pygame.K_UP:
				if cursor_y-1 >= 0:
					cursor_y -= 1
			elif event.key == pygame.K_LEFT:
				if cursor_x-1 >= 0:
					cursor_x -= 1
			elif event.key == pygame.K_RIGHT:
				if cursor_x+1 < GridCols:
					cursor_x += 1
			elif event.key == pygame.K_DOWN:
				if cursor_y+1 < GridRows:
					cursor_y += 1
			elif event.key == pygame.K_v:
				# draw closed list
				if drawmode == 0:
					drawmode = 1
				else:
					drawmode = 0
			elif event.key == pygame.K_a:		# -------- A* Search --------
				choice = -1
				while int(choice) < 1 or int(choice) > 6:
					choice = raw_input ("Enter (1) for Manhattan distance, (2) for Euclidean distance, (3) for Octile distance, (4) for Chebyshev distance, (5) for Straight-Diagonal Distance, or (6) Best/Minimum of all: ")
				MySearch = AStarSearch()
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y,choice)
				elapsed_time = time.time() - start_time
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_u:		# -------- Uniform Cost Search --------
				MySearch = UniformCostSearch()
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y,1)
				elapsed_time = time.time() - start_time
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_w:		# -------- Weighted A* Search --------
				choice = -1 #heuristic choice
				weight = 0 #weight of heuristic

				while (int(choice) < 1 or int(choice) > 6):
					choice = raw_input("Enter (1) for Manhattan distance, (2) for Euclidean Distance, (3) for Octile Distance, (4) for Chebyshev Distance, (5) for Straight-Diagonal Distance, or (6) Best/Minimum of all: ")
				while float(weight) < 1:
					weight = raw_input("Enter the selected weight for Weighted A* - must be >= 1: ")
				
				MySearch = WeightedAStarSearch(weight)
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y, choice)
				elapsed_time = time.time() - start_time
				
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_q:		# -------- Sequential A* Search --------
				MySearch = AStarSearch()

				
		#print(GridSurface,closed_list,final_path,path_cost,nodes_expanded,drawmode,elapsed_time,cell_costs, priority_list, heuristic_list)
		drawScreen(GridSurface,closed_list,final_path,path_cost,nodes_expanded,drawmode,elapsed_time,cell_costs, priority_list, heuristic_list)

pygame.quit()

first
Initially 79 32
I am here1 76 36
77 35
-----
I am here1 75 35
76 36
-----
I am here1 76 34
75 33
-----
I am here1 77 32
76 33
-----
I am here1 77 31
76 32
-----
I am here1 77 29
76 30
-----
I am here1 75 28
76 28
-----
I am here1 74 30
75 29
-----
I am here1 69 31
68 30
-----
I am here1 68 31
69 31
-----
I am here1 65 32
66 32
-----
I am here1 61 30
62 31
-----
I am here1 58 30
59 30
-----
I am here1 59 29
58 29
-----
I am here1 61 31
60 30
-----
I am here1 61 32
61 31
-----
I am here1 58 31
59 31
-----
I am here1 57 35
56 34
-----
I am here1 59 34
58 35
-----
I am here1 62 34
61 35
-----
I am here1 62 33
62 34
-----
I am here1 63 34
62 33
-----
I am here1 64 35
63 34
-----
I am here1 63 36
64 35
-----
I am here1 63 38
64 37
-----
I am here1 67 37
67 38
-----
I am here1 69 39
68 38
-----
I am here1 68 39
69 39
-----
I am here1 66 40
67 39
-----
I am here1 67 41
66 40
-----
I am here1 64 43
65 42
-----
I am here1 64 42
64 41
-----
I am here1 63 41
64 42
-----
I am here1 62 42
63 4

In [36]:
back_track(65,46)

65 46
5,5 ['5,4', '5,6', '6,5']
4,4 ['5,5']
80,33 ['79,32']
79,32 ['78,33']
78,33 ['77,34']
77,34 ['76,35']
76,35 ['77,35']
77,35 ['76,36']
76,36 ['75,35']
75,35 ['74,34']
74,34 ['75,33']
75,33 ['76,34']
76,34 ['76,33']
76,33 ['77,32']
77,32 ['76,32']
76,32 ['77,31']
77,31 ['76,30']
76,30 ['77,29']
77,29 ['76,28']
76,28 ['75,28']
75,28 ['75,29']
75,29 ['74,30']
74,30 ['73,30']
73,30 ['73,31']
73,31 ['72,30']
72,30 ['71,30']
71,30 ['70,31']
70,31 ['69,30']
69,30 ['68,29']
68,29 ['68,30']
68,30 ['69,31']
69,31 ['68,31']
68,31 ['67,31']
67,31 ['66,32']
66,32 ['65,32']
65,32 ['64,33']
64,33 ['65,34']
65,34 ['64,34']
64,34 ['63,33']
63,33 ['63,32']
63,32 ['62,32']
62,32 ['62,31']
62,31 ['61,30']
61,30 ['60,29']
60,29 ['59,30']
59,30 ['58,30']
58,30 ['58,29']
58,29 ['59,29']
59,29 ['60,30']
60,30 ['61,31']
61,31 ['61,32']
61,32 ['61,33']
61,33 ['60,32']
60,32 ['59,31']
59,31 ['58,31']
58,31 ['57,30']
57,30 ['57,29']
57,29 ['56,30']
56,30 ['57,31']
57,31 ['56,31']
56,31 ['57,32']
57,32 ['58,3

'65,46'

In [34]:
get_validate_neighbour(65,46)

(None, None)

In [35]:
validate_neighbour(65,46)

False

In [30]:
print(tree)

{'5,5': ['5,4', '5,6', '6,5'], '4,4': ['5,5'], '80,33': ['79,32'], '79,32': ['78,33'], '78,33': ['77,34'], '77,34': ['76,35'], '76,35': ['77,35'], '77,35': ['76,36'], '76,36': ['75,35'], '75,35': ['74,34'], '74,34': ['75,33'], '75,33': ['76,34'], '76,34': ['76,33'], '76,33': ['77,32'], '77,32': ['76,32'], '76,32': ['77,31'], '77,31': ['76,30'], '76,30': ['77,29'], '77,29': ['76,28'], '76,28': ['75,28'], '75,28': ['75,29'], '75,29': ['74,30'], '74,30': ['73,30'], '73,30': ['73,31'], '73,31': ['72,30'], '72,30': ['71,30'], '71,30': ['70,31'], '70,31': ['69,30'], '69,30': ['68,29'], '68,29': ['68,30'], '68,30': ['69,31'], '69,31': ['68,31'], '68,31': ['67,31'], '67,31': ['66,32'], '66,32': ['65,32'], '65,32': ['64,33'], '64,33': ['65,34'], '65,34': ['64,34'], '64,34': ['63,33'], '63,33': ['63,32'], '63,32': ['62,32'], '62,32': ['62,31'], '62,31': ['61,30'], '61,30': ['60,29'], '60,29': ['59,30'], '59,30': ['58,30'], '58,30': ['58,29'], '58,29': ['59,29'], '59,29': ['60,30'], '60,30': ['61